# ACEpotentials.jl + AtomsBase.jl Tutorial

## Introduction

This tutorial demonstrates how the ACEpotentials.jl package
interoperates with the AtomsBase.jl ecosystem.

## Installation

add and load general packages used in this notebook.

In [2]:
using Pkg
# Uncomment the next line if installing Julia for the first time
# Pkg.Registry.add("General")
# Pkg.Registry.add
Pkg.add(["ExtXYZ", "Unitful", "Distributed", "AtomsCalculators",
         "Molly", "AtomsCalculatorsUtilities", "AtomsBuilder",
         "AtomsBase", "LinearAlgebra",
         ])

# ACEpotentials installation:
# If ACEpotentials has not been installed yet, uncomment the following lines
# Add the ACE registry, which stores the ACEpotential package information
Pkg.Registry.add(RegistrySpec(url="https://github.com/ACEsuit/ACEregistry"))
Pkg.add(["GeomOpt", ])
# Pkg.add(["ACEpotentials",])

    Updating registry at `~/.julia/registries/ACE`
    Updating git-repo `https://github.com/ACEsuit/ACEregistry.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
      Compat entries added for 
  No Changes to `~/Documents/GitHub-2/ACESIDopt.jl/Project.toml`
  No Changes to `~/Documents/GitHub-2/ACESIDopt.jl/Manifest.toml`
Precompiling project...
  17006.8 ms  ✓ ACESIDopt
  1 dependency successfully precompiled in 19 seconds. 574 already precompiled.
     Cloning registry from "https://github.com/ACEsuit/ACEregistry"
Registry `ACE` already exists in `~/.julia/registries/ACE`.
   Resolving package versions...


Pkg.Resolve.ResolverError: Unsatisfiable requirements detected for package [38;5;13mGeomOpt [ca147568][39m:
 [38;5;13mGeomOpt [ca147568][39m log:
 ├─possible versions are: [38;5;13m0.0.1 - 0.1.0[39m or uninstalled
 ├─restricted to versions [38;5;13m*[39m by an explicit requirement, leaving only versions: [38;5;13m0.0.1 - 0.1.0[39m
 ├─restricted by compatibility requirements with [38;5;10mAtomsCalculators [a3e0e189][39m to versions: [38;5;13m0.0.5 - 0.1.0[39m or uninstalled, leaving only versions: [38;5;13m0.0.5 - 0.1.0[39m
 │ └─[38;5;10mAtomsCalculators [a3e0e189][39m log:
 │   ├─possible versions are: [38;5;10m0.1.0 - 0.2.3[39m or uninstalled
 │   └─restricted to versions [38;5;10m0.2.3 - 0.2[39m by [38;5;3mACESIDopt [8b8c1c5e][39m, leaving only versions: [38;5;10m0.2.3[39m
 │     └─[38;5;3mACESIDopt [8b8c1c5e][39m log:
 │       ├─possible versions are: [38;5;3m0.1.0[39m or uninstalled
 │       └─[38;5;3mACESIDopt [8b8c1c5e][39m is fixed to version [38

  We can check the status of the installed packages.

In [2]:
using Pkg; Pkg.activate(".")
Pkg.status()

  Activating project at `~/Documents/GitHub-2/ACESIDopt.jl`


Project ACESIDopt v0.1.0
Status `~/Documents/GitHub-2/ACESIDopt.jl/Project.toml`
  [3b96b61c] ACEpotentials v0.10.0
  [a963bdd2] AtomsBase v0.5.2
  [f5cc8831] AtomsBuilder v0.2.2
  [a3e0e189] AtomsCalculators v0.2.3
  [9855a07e] AtomsCalculatorsUtilities v0.1.8
  [352459e4] ExtXYZ v0.2.2
  [7073ff75] IJulia v1.33.0
  [b964fa9f] LaTeXStrings v1.4.0
  [98b081ad] Literate v2.21.0
⌅ [aa0f7f06] Molly v0.22.3
  [6f286f6a] MultivariateStats v0.10.3
  [91a5bcdd] Plots v1.41.3
⌅ [08abe8d2] PrettyTables v2.4.0
  [fd094767] Suppressor v0.2.8
  [1986cc42] Unitful v1.27.0
  [8ba89e20] Distributed v1.11.0
  [37e2e46d] LinearAlgebra v1.11.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


  Import all the packages that we will be using, create some processes
  for parallel model fitting

In [4]:
using ExtXYZ, Unitful, AtomsCalculators, Distributed, ACEpotentials,
      AtomsCalculatorsUtilities
using AtomsCalculatorsUtilities.SitePotentials: cutoff_radius
addprocs(10, exeflags="--project=$(Base.active_project())")
@everywhere using ACEpotentials

## Fit a potential for Cu

The tutorial can be adapted trivially to use datasets for Ni, Cu, Li, Mo, Si, Ge.

We generate a smallish model (about 300 basis functions) for Cu, using
correlation-order 3 (body-order 4), and default for rcut. Then we estimate
the model parameters using the standard BLR solver.

In [5]:
# generate a model for Cu
sym = :Cu
model = ace1_model(elements = [sym,], order = 3, totaldegree = [ 20, 16, 12 ])
@show length_basis(model)
@show cutoff_radius(model)
# estimate parameters
train, test, _ = ACEpotentials.example_dataset("Zuo20_$sym")
solver = ACEfit.BLR(; factorization = :svd)
acefit!(train, model;  solver=solver); GC.gc()
# quickly check test errors => 0.5 meV/atom and 0.014 eV/A are ok
ACEpotentials.compute_errors(test, model);

length_basis(model) = 319
cutoff_radius(model) = 6.5 Å
┌─────────┬──────────┬───────┬─────┬───────┬──────┐
│    Type │ #Configs │ #Envs │  #E │    #F │   #V │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│     nil │      262 │ 27416 │ 262 │ 82248 │    0 │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│   total │      262 │ 27416 │ 262 │ 82248 │    0 │
│ missing │        0 │     0 │   0 │     0 │ 1572 │
└─────────┴──────────┴───────┴─────┴───────┴──────┘


┌ Info: Assembling linear problem.
└ @ ACEfit /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/assemble.jl:28
┌ Info:   - Creating feature matrix with size (82510, 319).
└ @ ACEfit /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/assemble.jl:37
┌ Info:   - Beginning assembly with processor count:  11.
└ @ ACEfit /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/assemble.jl:40
Progress: 100%|█████████████████████████████████████████| Time: 0:02:56
┌ Info:   - Assembly completed.
└ @ ACEfit /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/assemble.jl:46
┌ Info: Assembling full weight vector.
└ @ ACEfit /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/assemble.jl:54
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
┌ Info: Entering bayesian_linear_regression_svd
└ @ ACEfit.BayesianLinear /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/bayesianlinear.jl:434
┌ Info: Computing SVD of (82510, 319) matrix
│   BLAS.get_num_threads() = 10
│   BLAS.get_config() = LBTConfig([ILP64]

Iter     Function value   Gradient norm 
     0     7.612064e+04     8.230760e+04
 * time: 0.009598016738891602
     1     1.745505e+04     1.676056e+05
 * time: 0.41065311431884766
     2    -2.245967e+05     2.130711e+06
 * time: 0.41126108169555664
     3    -2.317119e+05     1.299648e+06
 * time: 0.41289210319519043
     4    -2.326797e+05     1.184333e+06
 * time: 0.413485050201416
     5    -2.332761e+05     7.624167e+04
 * time: 0.4139540195465088
     6    -2.332796e+05     1.102229e+04
 * time: 0.41454100608825684
     7    -2.332925e+05     1.489574e+05
 * time: 0.4155240058898926
     8    -2.365940e+05     1.689311e+06
 * time: 0.4200770854949951
     9    -2.384742e+05     8.880206e+05
 * time: 0.42743492126464844
    10    -2.395635e+05     1.205582e+05
 * time: 0.4305140972137451
    11    -2.405116e+05     1.007096e+06
 * time: 0.4764981269836426
    12    -2.412142e+05     4.527475e+05
 * time: 0.4773399829864502
    13    -2.416141e+05     2.838910e+05
 * time: 0.4786

┌ Info: Optimization complete
│   Results =  * Status: success

 * Candidate solution
    Final objective value:     -2.425484e+05

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 9.20e-10 ≤ 1.0e-08
    |x - x'|/|x'|          = 1.91e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 7.45e-10 ≰ 0.0e+00

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    23
    f(x) calls:    114
    ∇f(x) calls:   114

└ @ ACEfit.BayesianLinear /Users/msachs2/.julia/packages/ACEfit/IRtHL/src/bayesianlinear.jl:487


┌──────┬─────────┬──────────┬─────────┐
│ Type │ E [meV] │ F [eV/A] │ V [meV] │
├──────┼─────────┼──────────┼─────────┤
│  nil │   0.502 │    0.014 │   0.000 │
├──────┼─────────┼──────────┼─────────┤
│  set │   0.502 │    0.014 │   0.000 │
└──────┴─────────┴──────────┴─────────┘


┌ Info: RMSE Table
└ @ ACEpotentials /Users/msachs2/.julia/packages/ACEpotentials/nR4od/src/atoms_data.jl:378


┌──────┬─────────┬──────────┬─────────┐
│ Type │ E [meV] │ F [eV/A] │ V [meV] │
├──────┼─────────┼──────────┼─────────┤
│  nil │   0.291 │    0.007 │   0.000 │
├──────┼─────────┼──────────┼─────────┤
│  set │   0.291 │    0.007 │   0.000 │
└──────┴─────────┴──────────┴─────────┘


┌ Info: MAE Table
└ @ ACEpotentials /Users/msachs2/.julia/packages/ACEpotentials/nR4od/src/atoms_data.jl:383


## Geometry Optimization with GeomOpt

( Note: we should use GeometryOptimization.jl, but this is not yet updated to
  AtomsBase.jl version 0.4. )

First import some stuff + a little hack to make GeomOpt play nice with
the latest AtomsBase. This is a shortcoming of DecoratedParticles.jl
and requires some updates to fully implement the AtomsBase interface.

In [ ]:
using AtomsBuilder, GeomOpt, AtomsCalculators, AtomsBase
using AtomsBase: FlexibleSystem, FastSystem
using AtomsCalculators: potential_energy
function _flexiblesystem(sys)
   c3ll = cell(sys)
   particles = [ AtomsBase.Atom(species(sys, i), position(sys, i))
                 for i = 1:length(sys) ]
   return FlexibleSystem(particles, c3ll)
end;

We generate a cubic Cu unit cell, but our potential might not have the same
equilibrium bond distance as the default in AtomsBuilder, so we optimize
the unit cell.

In [ ]:
ucell = bulk(sym, cubic=true)
ucell, _ = GeomOpt.minimise(ucell, model; variablecell=true)

We keep the energy of the equilibrated unit cell to later compute the
defect formation energy.

In [ ]:
Eperat = potential_energy(ucell, model) / length(ucell)
@show Eperat;

Now that we have an equilibrated unit cell we enlarge it, and then delete
an atom to generate a vacancy defect.

In [ ]:
sys = _flexiblesystem(ucell) * (2,2,2)
deleteat!(sys, 1)
sys

Now we do another geometry optimization to get the equilibrium geometry.

In [ ]:
vacancy_equil, result = GeomOpt.minimise(sys, model; variablecell = false)
@show result.g_residual;

We get an estimate of the formation energy. Note this is likely a poor
estimate since we didn't train the model on vacancy configurations.

In [ ]:
E_def = potential_energy(vacancy_equil, model) - length(sys) * Eperat
@show E_def;

One of the most useful Julia packages is ForwardDiff. This is used in
AtomsCalculatorsUtilities to implement hessians. ACEpotentials.jl only
has to ensure that its site energies allow Dual numbers as inputs.

In [ ]:
using AtomsCalculatorsUtilities.SitePotentials: hessian
using LinearAlgebra: Symmetric, eigvals

H = ustrip.( hessian(vacancy_equil, model) )
Λ = eigvals(Symmetric(H))
S_vib = sum(λ -> λ > 1e-10 ? log(λ) : 0, Λ)

## Molecular Dynamics with Molly

In [ ]:
import Molly
sys = rattle!(bulk(sym, cubic=true) * (2,2,2), 0.03)
sys_md = Molly.System(sys; force_units=u"eV/Å", energy_units=u"eV")
temp = 298.0u"K"
sys_md = Molly.System(
   sys_md;
   general_inters = (model,),
   velocities = Molly.random_velocities(sys_md, temp),
   loggers=(temp = Molly.TemperatureLogger(100),
          energy = Molly.PotentialEnergyLogger(typeof(1.0u"eV"), 100),), )
simulator = Molly.VelocityVerlet(
   dt = 1.0u"fs",
   coupling = Molly.AndersenThermostat(temp, 3.0u"fs"), )

Molly.simulate!(sys_md, simulator, 1000)

# looks like a reasonably stable simulation.
@info("Temperature history:", sys_md.loggers.temp.history)
@info("Energy history:", sys_md.loggers.energy.history)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*